In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"],1)
application_df.head()

C:\Users\tamik\AppData\Local\Temp\ipykernel_7328\1309730736.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(["EIN", "NAME"],1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_count =application_df["APPLICATION_TYPE"].value_counts()
application_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_count[application_type_count<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_type_count =application_df["CLASSIFICATION"].value_counts()
classification_type_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_type_count[classification_type_count<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Look at ASK_AMT value counts for binning
ASK_AMT_count =application_df["ASK_AMT"].value_counts()
ASK_AMT_count

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [9]:
bins = [0, 5000, 10000, 100000, 1000000, 10000000, 100000000, float('inf')]
bin_labels = ['0-4999', '5000-9999', '10000-99999', '100000-999999', 
              '1000000-9999999', '10000000-99999999', '>999999999']
application_df['ASK_AMT_RANGE'] = pd.cut(application_df['ASK_AMT'],
                                       bins=bins, labels=bin_labels, include_lowest=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ASK_AMT_RANGE
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,0-4999
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,100000-999999
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,0-4999
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,5000-9999
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,100000-999999
...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,0-4999
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,0-4999
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,0-4999
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,0-4999


In [10]:
application_df = application_df.drop(["ASK_AMT"],1)
application_df.head()

C:\Users\tamik\AppData\Local\Temp\ipykernel_7328\852938603.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(["ASK_AMT"],1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_RANGE
0,T10,Independent,C1000,ProductDev,Association,1,0,N,1,0-4999
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1,100000-999999
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,0,0-4999
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1,5000-9999
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1,100000-999999


In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_RANGE_0-4999,ASK_AMT_RANGE_5000-9999,ASK_AMT_RANGE_10000-99999,ASK_AMT_RANGE_100000-999999,ASK_AMT_RANGE_1000000-9999999,ASK_AMT_RANGE_10000000-99999999,ASK_AMT_RANGE_>999999999
0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4000      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6461 (25.24 KB)
Trainable params: 6461 (25.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_weights_opt_1.h5",  
    save_weights_only=True,       
    save_best_only=False,         
    save_freq=5)   

fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5704 - accuracy: 0.7186
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5545 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5509 - accuracy: 0.7309
Epoch 4/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5482 - accuracy: 0.7306
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5469 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5459 - accuracy: 0.7331
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5449 - accuracy: 0.7329
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5443 - accuracy: 0.7329
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5435 - accuracy: 0.7335
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5429 - accura

804/804 [==============================] - 3s 3ms/step - loss: 0.5276 - accuracy: 0.7430
Epoch 82/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5273 - accuracy: 0.7434
Epoch 83/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5275 - accuracy: 0.7428
Epoch 84/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5273 - accuracy: 0.7442
Epoch 85/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5275 - accuracy: 0.7432
Epoch 86/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5268 - accuracy: 0.7441
Epoch 87/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5273 - accuracy: 0.7438
Epoch 88/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5269 - accuracy: 0.7446
Epoch 89/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5272 - accuracy: 0.7425
Epoch 90/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5267 - accuracy: 

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5674 - accuracy: 0.7256 - 489ms/epoch - 2ms/step
Loss: 0.5674080848693848, Accuracy: 0.7255976796150208


In [18]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_opt1.h5')

C:\Users\tamik\anaconda3\envs\bootcamp3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
